In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)


In [3]:
# load dataset
dataframe = pandas.read_csv('iris.csv', header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]


In [4]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [5]:
from keras.utils import np_utils
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [6]:
dummy_y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [7]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(8, activation='relu', input_shape=(4,)))
    model.add(Dense(3, activation = 'softmax'))
    # Compile model
    model.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])
    return model


In [12]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=200, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, dummy_y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'multilabel-indicator' instead.

<h2> Tuning Layers and Number of Neurons in The Model
<h3> Evaluate a Smaller Network

In [44]:
from keras import layers
def keras_model():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(4,)))
    model.add(layers.Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(Dense(3, activation = 'softmax'))
    # Compile model
    model.compile(optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy'])
    return model


In [62]:

import numpy as np
k = 10
num_val_samples = len(X) // k
num_epochs = 200
all_scores = []
for i in range(k):
    
    print('processing fold #', i)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = dummy_y[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(
    [X[:i * num_val_samples],
    X[(i + 1) * num_val_samples:]],
    axis=0)
    
    partial_train_targets = np.concatenate(
    [dummy_y[:i * num_val_samples],
    dummy_y[(i + 1) * num_val_samples:]],
    axis=0)
    model = keras_model()
    model.fit(partial_train_data, partial_train_targets,
    epochs=num_epochs, batch_size=5, verbose=1)
    val_loss, val_acc = model.evaluate(val_data, val_targets, verbose=1)
    all_scores.append(val_acc)

processing fold # 0
Epoch 1/200
135/135 [==============================] - 12s 85ms/step - loss: 3.2129 - acc: 0.2593
Epoch 2/200
135/135 [==============================] - 0s 1ms/step - loss: 2.5171 - acc: 0.2444
Epoch 3/200
135/135 [==============================] - 0s 1ms/step - loss: 1.9416 - acc: 0.3185
Epoch 4/200
135/135 [==============================] - 0s 1ms/step - loss: 1.6960 - acc: 0.3259
Epoch 5/200
135/135 [==============================] - 0s 1ms/step - loss: 1.4807 - acc: 0.2741
Epoch 6/200
135/135 [==============================] - 0s 1ms/step - loss: 1.4266 - acc: 0.2741
Epoch 7/200
135/135 [==============================] - 0s 1ms/step - loss: 1.2837 - acc: 0.2741
Epoch 8/200
135/135 [==============================] - 0s 1ms/step - loss: 1.2666 - acc: 0.2593
Epoch 9/200
135/135 [==============================] - 0s 2ms/step - loss: 1.2022 - acc: 0.3259
Epoch 10/200
135/135 [==============================] - 0s 2ms/step - loss: 1.1680 - acc: 0.3037
Epoch 11/200
135/

135/135 [==============================] - 0s 1ms/step - loss: 0.1767 - acc: 0.8963
Epoch 169/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2268 - acc: 0.9185
Epoch 170/200
135/135 [==============================] - 0s 1ms/step - loss: 0.1868 - acc: 0.9333
Epoch 171/200
135/135 [==============================] - 0s 1ms/step - loss: 0.1975 - acc: 0.9185
Epoch 172/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2009 - acc: 0.9037
Epoch 173/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2050 - acc: 0.8741
Epoch 174/200
135/135 [==============================] - 0s 1ms/step - loss: 0.1393 - acc: 0.9556
Epoch 175/200
135/135 [==============================] - 0s 1ms/step - loss: 0.1792 - acc: 0.9333
Epoch 176/200
135/135 [==============================] - 0s 1ms/step - loss: 0.1425 - acc: 0.9407
Epoch 177/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2021 - acc: 0.9037
Epoch 178/200
135/135 [===========

135/135 [==============================] - 0s 999us/step - loss: 0.2786 - acc: 0.9111
Epoch 135/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2216 - acc: 0.9185
Epoch 136/200
135/135 [==============================] - 0s 955us/step - loss: 0.2334 - acc: 0.9481
Epoch 137/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2721 - acc: 0.9185
Epoch 138/200
135/135 [==============================] - 0s 940us/step - loss: 0.2031 - acc: 0.9630
Epoch 139/200
135/135 [==============================] - 0s 903us/step - loss: 0.3366 - acc: 0.8889
Epoch 140/200
135/135 [==============================] - 0s 960us/step - loss: 0.2286 - acc: 0.9333
Epoch 141/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2587 - acc: 0.9185
Epoch 142/200
135/135 [==============================] - 0s 938us/step - loss: 0.2733 - acc: 0.8889
Epoch 143/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2477 - acc: 0.9185
Epoch 144/200
135/135 

135/135 [==============================] - 0s 1ms/step - loss: 0.4552 - acc: 0.7556
Epoch 100/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3984 - acc: 0.8519
Epoch 101/200
135/135 [==============================] - 0s 955us/step - loss: 0.4193 - acc: 0.8296
Epoch 102/200
135/135 [==============================] - 0s 1ms/step - loss: 0.4543 - acc: 0.7926
Epoch 103/200
135/135 [==============================] - 0s 1ms/step - loss: 0.4119 - acc: 0.8370
Epoch 104/200
135/135 [==============================] - 0s 900us/step - loss: 0.4008 - acc: 0.8593
Epoch 105/200
135/135 [==============================] - 0s 949us/step - loss: 0.4605 - acc: 0.7852
Epoch 106/200
135/135 [==============================] - 0s 1ms/step - loss: 0.4073 - acc: 0.8074
Epoch 107/200
135/135 [==============================] - 0s 899us/step - loss: 0.4308 - acc: 0.8222
Epoch 108/200
135/135 [==============================] - 0s 904us/step - loss: 0.4096 - acc: 0.8370
Epoch 109/200
135/135 [=

135/135 [==============================] - 0s 1ms/step - loss: 0.3484 - acc: 0.8593
Epoch 183/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2994 - acc: 0.8741
Epoch 184/200
135/135 [==============================] - 0s 988us/step - loss: 0.3062 - acc: 0.8444
Epoch 185/200
135/135 [==============================] - 0s 999us/step - loss: 0.3146 - acc: 0.8667
Epoch 186/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3106 - acc: 0.8963
Epoch 187/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2947 - acc: 0.8963
Epoch 188/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3758 - acc: 0.8370
Epoch 189/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3143 - acc: 0.8296
Epoch 190/200
135/135 [==============================] - 0s 927us/step - loss: 0.3257 - acc: 0.8519
Epoch 191/200
135/135 [==============================] - 0s 997us/step - loss: 0.3973 - acc: 0.8370
Epoch 192/200
135/135 [===

...

135/135 [==============================] - 0s 1ms/step - loss: 0.3133 - acc: 0.8370
Epoch 94/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3564 - acc: 0.8222
Epoch 95/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3718 - acc: 0.8222
Epoch 96/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3424 - acc: 0.8593
Epoch 97/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3344 - acc: 0.8222
Epoch 98/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3320 - acc: 0.8444
Epoch 99/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3791 - acc: 0.8000
Epoch 100/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2984 - acc: 0.8593
Epoch 101/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3148 - acc: 0.8296
Epoch 102/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3598 - acc: 0.8148
Epoch 103/200
135/135 [=================

135/135 [==============================] - 0s 1ms/step - loss: 0.3669 - acc: 0.8370
Epoch 60/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3413 - acc: 0.8148
Epoch 61/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3073 - acc: 0.8815
Epoch 62/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3533 - acc: 0.8593
Epoch 63/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3337 - acc: 0.8074
Epoch 64/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3554 - acc: 0.8296
Epoch 65/200
135/135 [==============================] - 0s 846us/step - loss: 0.3449 - acc: 0.8370
Epoch 66/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3389 - acc: 0.8370
Epoch 67/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3112 - acc: 0.8667
Epoch 68/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3374 - acc: 0.8222
Epoch 69/200
135/135 [===================

Epoch 25/200
135/135 [==============================] - 0s 1ms/step - loss: 0.6683 - acc: 0.6222
Epoch 26/200
135/135 [==============================] - 0s 1ms/step - loss: 0.6020 - acc: 0.6889
Epoch 27/200
135/135 [==============================] - 0s 1ms/step - loss: 0.6523 - acc: 0.6370
Epoch 28/200
135/135 [==============================] - 0s 1ms/step - loss: 0.6486 - acc: 0.6222
Epoch 29/200
135/135 [==============================] - 0s 1ms/step - loss: 0.5957 - acc: 0.6963
Epoch 30/200
135/135 [==============================] - 0s 1ms/step - loss: 0.5755 - acc: 0.7111
Epoch 31/200
135/135 [==============================] - 0s 1ms/step - loss: 0.5681 - acc: 0.7037
Epoch 32/200
135/135 [==============================] - 0s 1ms/step - loss: 0.5706 - acc: 0.6815
Epoch 33/200
135/135 [==============================] - 0s 1ms/step - loss: 0.5626 - acc: 0.7481
Epoch 34/200
135/135 [==============================] - 0s 1ms/step - loss: 0.5699 - acc: 0.7111
Epoch 35/200
135/135 [========

Epoch 109/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2591 - acc: 0.8815
Epoch 110/200
135/135 [==============================] - 0s 999us/step - loss: 0.2231 - acc: 0.9185
Epoch 111/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2132 - acc: 0.9185
Epoch 112/200
135/135 [==============================] - 0s 1ms/step - loss: 0.1812 - acc: 0.9259
Epoch 113/200
135/135 [==============================] - 0s 1ms/step - loss: 0.3341 - acc: 0.8074
Epoch 114/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2374 - acc: 0.8963
Epoch 115/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2787 - acc: 0.8741
Epoch 116/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2255 - acc: 0.9037
Epoch 117/200
135/135 [==============================] - 0s 957us/step - loss: 0.2428 - acc: 0.9037
Epoch 118/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2472 - acc: 0.8963
Epoch 119/200
13

135/135 [==============================] - 0s 1ms/step - loss: 0.1717 - acc: 0.9185
Epoch 193/200
135/135 [==============================] - 0s 1000us/step - loss: 0.2275 - acc: 0.8963
Epoch 194/200
135/135 [==============================] - 0s 1ms/step - loss: 0.2076 - acc: 0.9185
Epoch 195/200
135/135 [==============================] - 0s 936us/step - loss: 0.2302 - acc: 0.8889
Epoch 196/200
135/135 [==============================] - 0s 1ms/step - loss: 0.1817 - acc: 0.9185
Epoch 197/200
135/135 [==============================] - 0s 1ms/step - loss: 0.1796 - acc: 0.9185
Epoch 198/200
135/135 [==============================] - 0s 1ms/step - loss: 0.1994 - acc: 0.8963
Epoch 199/200
135/135 [==============================] - 0s 965us/step - loss: 0.1467 - acc: 0.9481
Epoch 200/200
15/15 [==============================] - 4s 236ms/step


In [63]:
all_scores

[1.0,
 1.0,
 1.0,
 1.0,
 0.800000011920929,
 0.9333333373069763,
 1.0,
 0.9333333373069763,
 0.9333333373069763,
 1.0]

In [64]:
np.mean(all_scores)

0.9600000023841858